In [8]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn import metrics
from sklearn.metrics import explained_variance_score
from sklearn.metrics import max_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import median_absolute_error
from sklearn.metrics import r2_score
import math
import time

In [9]:
#df_train = pd.read_csv('trainCO.txt', header=None)
#df_test = pd.read_csv('testCO.txt', header=None)

df_train = pd.read_csv('/Users/ivanizonin/izonin/sgtm/Datasets/Jinsheng2_train.txt', header=None)
df_test = pd.read_csv('/Users/ivanizonin/izonin/sgtm/Datasets/Jinsheng2_test.txt', header=None)

#df_train = pd.read_csv('/Users/ivanizonin/izonin/sgtm/Datasets/Nzone1_train.txt', header=None)
#df_test = pd.read_csv('/Users/ivanizonin/izonin/sgtm/Datasets/Nzone1_test.txt', header=None)

#df_train = pd.read_csv('/Users/ivanizonin/izonin/sgtm/Datasets/Resilience1Train.txt', header=None)
#df_test = pd.read_csv('/Users/ivanizonin/izonin/sgtm/Datasets/Resilience1Test.txt', header=None)

#df_train = pd.read_csv('/Users/ivanizonin/izonin/sgtm/Datasets/StrengthTrain.txt', header=None)
#df_test = pd.read_csv('/Users/ivanizonin/izonin/sgtm/Datasets/StrengthTest.txt', header=None)

train_size = len(df_train)

train_X = df_train.iloc[:, :-1]
train_y = df_train.iloc[:,-1]
test_X = df_test.iloc[:, :-1]
test_y = df_test.iloc[:,-1]


In [10]:
scaler = MaxAbsScaler()
scaler.fit(train_X)
train_X = scaler.transform(train_X)
test_X = scaler.transform(test_X)

In [11]:
# поліноміальне розширення простору вхідних даних задачі
poly = PolynomialFeatures(2)
train_X = poly.fit_transform(train_X)
test_X = poly.fit_transform(test_X)
#np.savetxt("C:/Users/User/Desktop/Публікації/graps1.txt", train_X)
#np.savetxt("C:/Users/User/Desktop/Публікації/graps2.txt", test_X)

In [12]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.utils.multiclass import unique_labels
from sklearn.metrics import euclidean_distances
class GTM(BaseEstimator, ClassifierMixin):

    def __init__(self, name = "GTM", numb_of_steps = 1000, n_iter = 50, center_of_mass = False):
        self.name = name
        self.numb_of_steps = numb_of_steps
        self.n_iter = n_iter
        self.center_of_mass = center_of_mass
        

    def fit(self, X, y):
        # Check that X and y have correct shape
        X, y = check_X_y(X, y)
        
        start_time = time.time()
        
        m = X.shape[1]
        step = 0
        
        if self.numb_of_steps > m:
            print("Number of steps should be lower then features count (" + str(m) +")")
        print(m)
        
        centered_X = X
        centered_y = y
        
        com_x = np.zeros(m)
        com_y = 0 #np.zeros(y.shape[1])
        
        if self.center_of_mass:
            com_x = np.mean(X, axis=0)
            com_y = np.mean(y)
            centered_X = X - com_x
            centered_y = y - com_y
        
        
        self.com_x = com_x
        self.com_y = com_y
           
        self.basic_instances = []
        self.b_instances_x = []
        self.b_instances_y = []
        
    
        while step < m:
            iter = 0
            
            #TODO change
            i = np.argmax(np.sum(centered_X**2, axis=1))
            
            initial_instance = centered_X[i,]
            
            basic_instance = initial_instance
            
            while iter < self.n_iter:
                vector_relations = np.sum(np.multiply(centered_X, basic_instance), axis=1) \
                / np.sum(basic_instance**2)
                basic_instance = np.sum(np.multiply(centered_X, vector_relations[:, np.newaxis]), axis=0) \
                / np.sum(vector_relations**2)
                iter = iter + 1
                
            k1_values = np.sum(np.multiply(centered_X, basic_instance), axis=1) \
            / np.sum(basic_instance**2, axis=0)
            
            k2_values = np.sum(np.multiply(centered_X, basic_instance), axis=1) \
            / math.sqrt(np.sum(basic_instance**2, axis=0))
            
            if step == self.numb_of_steps:
                self.num_steps = step
                print("Break, step = " + str(step))
                break
                
            b_instance_x = np.sum(np.multiply(centered_X, k1_values[:, np.newaxis]), axis=0) \
            / np.sum(k1_values**2, axis=0)
            
    
            b_instance_y = np.sum(np.multiply(centered_y, k1_values), axis=0) \
            / np.sum(k1_values**2, axis=0)
            
    
            centered_X = np.subtract(centered_X, np.multiply(k1_values[:, np.newaxis], b_instance_x))
            centered_y = np.subtract(centered_y, np.multiply(k1_values, b_instance_y))

            
            self.basic_instances.append(basic_instance)
            self.b_instances_x.append(b_instance_x)
            self.b_instances_y.append(b_instance_y)
            
            step = step + 1
            print("Step: " + str(step))
            
        self.num_steps = step
        print("--- %s seconds ---" % (time.time() - start_time))
        
        return self

    def predict(self, X):

        # Check is fit had been called
        #check_is_fitted(self, ['X_', 'y_'])

        # Input validation
        X = check_array(X)
        
        centered_X = X
        
        if self.center_of_mass:
            com_x = self.com_x
            centered_X = X - com_x
        
        k1_values_a = []
        
        for step in range(0, self.num_steps):
            k1_values = np.sum(np.multiply(centered_X, self.basic_instances[step]), axis=1) \
            / np.sum(self.basic_instances[step]**2, axis=0)
            centered_X = np.subtract(centered_X, np.multiply(k1_values[:, np.newaxis], self.b_instances_x[step]))
            k1_values_a.append(k1_values)
        
        y = np.zeros(X.shape[0])
        
        for step in reversed(range(0, self.num_steps)):
            y = np.multiply(k1_values_a[step], self.b_instances_y[step]) + y
            
        y = y + self.com_y
        
        return y

In [13]:
gtm = GTM()
# для зменшення кількості нейронів в прихованому шарі
#gtm = GTM(numb_of_steps = 2)
# gtm = GTM(center_of_mass = False)
# навчання
 # print('Training model..')
start_time = time.time()
gtm.fit(train_X, train_y)
stop_time = time.time()

# застосування
train_pred_y = gtm.predict(train_X)
#print(test_X)
pred_y = gtm.predict(test_X)

#print(pred_y)

2
Step: 1
Step: 2
--- 0.003283977508544922 seconds ---


In [14]:
print('Training erros:')
print("MaxError: " + str(max_error(train_y, train_pred_y)))
print("MAE: " + str(mean_absolute_error(train_y, train_pred_y)))
print("MSE: " + str(mean_squared_error(train_y, train_pred_y)))
print("MedError: " + str(median_absolute_error(train_y, train_pred_y)))
print("RMSE: " + str(mean_squared_error(train_y, train_pred_y, squared=False)))
print("MAPE: " + str(mean_absolute_percentage_error(train_y, train_pred_y)))
print("R2: " + str(r2_score(train_y, train_pred_y)))
print("Training time""=%s seconds" % (time.time() - start_time))
print("/")
print('Testing errors:')
print("MaxError: " + str(max_error(test_y, pred_y)))
print("MAE: " + str(mean_absolute_error(test_y, pred_y)))
print("MSE: " + str(mean_squared_error(test_y, pred_y)))
print("MedError: " + str(median_absolute_error(test_y, pred_y)))
print("RMSE: " + str(mean_squared_error(test_y, pred_y, squared=False)))
print("MAPE: " + str(mean_absolute_percentage_error(test_y, pred_y)))
print("R2: " + str(r2_score(test_y, pred_y)))
print("/")
print(pred_y)
#np.savetxt('C:/Users/User/Desktop/Публікації/DSMP/Стаття/Medical Cost Personal Datasets/Results/TEST+WP.csv', test_y, delimiter=',', fmt='%1.5f')


Training erros:
MaxError: 11.807577639812678
MAE: 2.4234816131692156
MSE: 10.523282068628871
MedError: 1.6274876613389788
RMSE: 3.243960861143191
MAPE: 612923321378091.9
R2: 0.703019730414066
Training time=0.012542963027954102 seconds
/
Testing errors:
MaxError: 5.968009980868089
MAE: 2.15923546521435
MSE: 7.164291534582908
MedError: 1.6700309344954312
RMSE: 2.676619422813581
MAPE: 1.0127870616452121
R2: 0.7306131243641083
/
[ 6.63478337  0.93342265 12.3361441  -0.84447165 10.5582498   4.85688908
 11.96846626 -3.3425979   8.06012355  4.50433496 11.61591214  1.6536546
  8.76523178  5.20944319 16.61216464 -2.17789237  9.22482908  3.52346836
 10.63504554  1.74557406]
